# Examples

The below illustrates the way this code can be used to accomplish various common tasks. 

**Outputs can be written to a CSV file, a SQLite database, or both**. In the case of a SQLite database, if it does not yet exist, it will be automatically created, set up, and populated, so no experience with SQLite is necessary (until you want to query it for data, that is). One benefit of using the SQLite database is that unique indexes ensure that no duplicates are inserted. We handle this with CSVs less efficiently, by scanning over the CSV for duplicate IDs after we're done with the full output. 

The reason we have to work around duplicates at all is because of how the Regulations.gov API's pagination mechanism works (the reason: you get up to 20 pages per "query", each of which has up to 250 items. If you need more than 250x20 = 5000 items, you have to use a second "query", which is the same as the first, but filtered to a lastModifiedDate >= the maximum lastModifiedDate of the previous query. As a result, there will be repeated items. Unfortunately, this is necessary because there may be multiple items with the same lastModifiedDate).

# Get an instance of the CommentDownloader

**This is required for all examples below**. You must have an API key, which you can get at https://open.gsa.gov/api/regulationsgov/#getting-started. The "DEMO_KEY" below is functional, but is limited to only 25 requests per hour, while a normal API key has up to 1,000 requests per hour.

In [1]:
from comments_downloader import CommentsDownloader
downloader = CommentsDownloader(api_key="DEMO_KEY")

Alternatively, if you don't want to hard-code your API key, you could do:

    import getpass
    from comments_downloader import CommentsDownloader
    
    api_key = getpass.getpass("Enter API key")
    downloader = CommentsDownloader(api_key)

# Q: How do I download all comments associated with a docket?

In [2]:
# We are outputting to both the database and the CSV here, but you can exclude either the db_filename or 
#     csv_filename parameters if you want to export to only one of them.
# Notice that the database is created automatically. If the database already existed, we would be appending to it, not replacing it.

downloader.gather_comments_by_docket("FDA-2021-N-0270", db_filename="my_database.db", csv_filename="my_csv.csv")

2021-05-27 12:02:40: Getting documents associated with docket FDA-2021-N-0270...

Found 1 documents...
2021-05-27 12:02:41: Inserting 1 records into database...
2021-05-27 12:02:41: Writing 1 records to document_headers_120240.csv...Done
2021-05-27 12:02:41: Removing any duplicates in the CSV...
2021-05-27 12:02:41: Done. Removed 0 duplicate rows from document_headers_120240.csv.
2021-05-27 12:02:41: Finished: approximately 1 documents collected

Done----------------

******************************
2021-05-27 12:02:41: Getting comments for document FDA-2021-N-0270-0001...

2021-05-27 12:02:41: Getting objectId for document FDA-2021-N-0270-0001...Got it (0900006484a930da)
2021-05-27 12:02:41: Getting comment headers associated with document FDA-2021-N-0270-0001...

Found 181 comments...
2021-05-27 12:02:42: Inserting 181 records into database...
2021-05-27 12:02:42: Writing 181 records to comment_headers_120241.csv...Done
2021-05-27 12:02:42: Removing any duplicates in the CSV...
2021-0

## Look at the data

In [3]:
# look at the data

# See readme for database schema. The comment details (including its plain-text) are in comments_detail
import pandas as pd
import sqlite3

conn = sqlite3.connect("my_database.db")
pd.read_sql_query("select * from comments_detail limit 5", conn)

,commentId,agencyId,category,city,comment,commentOn,commentOnDocumentId,country,docAbstract,docketId,...,restrictReasonType,stateProvinceRegion,submitterRep,submitterRepCityState,subtype,title,trackingNbr,withdrawn,zip,sqltime
0,FDA-2021-N-0270-0077,FDA,Drug Association - D0012,None,I have been T1D for 60 yrs. now. After about 4...,0900006484a930da,FDA-2021-N-0270-0001,United States,None,FDA-2021-N-0270,...,None,TX,None,None,Electronic Regulation from Form,Comment from Ann M Jenschke,kol-px4s-8ss3,0,None,2021-05-27 16:03:21
1,FDA-2021-N-0270-0151,FDA,None,None,I am writing to support a recommendation to ap...,0900006484a930da,FDA-2021-N-0270-0001,United States,None,FDA-2021-N-0270,...,None,CA,None,None,Electronic Regulation from Form,Comment from Kevin O'Scanlon,kp4-bmwl-0drg,0,None,2021-05-27 16:03:21
2,FDA-2021-N-0270-0052,FDA,Individual Consumer,None,As a mom to identical twin boys and wife to a ...,0900006484a930da,FDA-2021-N-0270-0001,United States,None,FDA-2021-N-0270,...,None,CA,None,None,Electronic Regulation from Form,Comment from Susan Sjolund,kok-fogq-kwux,0,None,2021-05-27 16:03:21
3,FDA-2021-N-0270-0113,FDA,Drug Industry - C0022,None,As a parent of one child who is a Type 1 diabe...,0900006484a930da,FDA-2021-N-0270-0001,United States,None,FDA-2021-N-0270,...,None,TX,None,None,Electronic Regulation from Form,Comment from Jen Meisenheimer,kov-xxym-w9cm,0,None,2021-05-27 16:03:21
4,FDA-2021-N-0270-0065,FDA,Individual Consumer,None,I was diagnosed with Type 1 diabetes when I wa...,0900006484a930da,FDA-2021-N-0270-0001,United States,None,FDA-2021-N-0270,...,None,CA,None,None,Electronic Regulation from Form,Comment from Lona Albano,kok-tye2-0e33,0,None,2021-05-27 16:03:21


In [4]:
# look at header information for comments
pd.read_sql_query("select * from comments_header limit 5", conn)

,commentId,agencyId,documentType,lastModifiedDate,objectId,postedDate,title,withdrawn,sqltime
0,FDA-2021-N-0270-0002,FDA,Public Submission,2021-04-01T20:31:47Z,0900006484a942e8,2021-04-01T04:00:00Z,Comment from Mark Atkinson,0,2021-05-27 16:02:42
1,FDA-2021-N-0270-0003,FDA,Public Submission,2021-04-12T16:29:37Z,0900006484aa0b69,2021-04-12T04:00:00Z,Comment from Bodie Francis,0,2021-05-27 16:02:42
2,FDA-2021-N-0270-0004,FDA,Public Submission,2021-04-12T16:31:21Z,0900006484aa0bf7,2021-04-12T04:00:00Z,Comment from THOMAS WEGESIN,0,2021-05-27 16:02:42
3,FDA-2021-N-0270-0005,FDA,Public Submission,2021-04-20T19:42:30Z,0900006484aa5c46,2021-04-20T04:00:00Z,Comment from Ginger Patterson,0,2021-05-27 16:02:42
4,FDA-2021-N-0270-0006,FDA,Public Submission,2021-04-20T19:46:27Z,0900006484aad502,2021-04-20T04:00:00Z,Comment from Kathryn Dancy,0,2021-05-27 16:02:42


In [5]:
# look at header information for the docket's documents. Note that we don't download the documents' details in this process
pd.read_sql_query("select * from documents_header limit 5", conn)

,documentId,commentEndDate,commentStartDate,docketId,documentType,frDocNum,lastModifiedDate,objectId,postedDate,subtype,title,withdrawn,sqltime
0,FDA-2021-N-0270-0001,2021-05-27T03:59:59Z,2021-04-01T04:00:00Z,FDA-2021-N-0270,Notice,2021-06708,2021-05-27T01:00:47Z,0900006484a930da,2021-04-01T04:00:00Z,Advisory Committee,Endocrinologic and Metabolic Drugs Advisory Co...,0,2021-05-27 16:02:41


In [6]:
conn.close()

# Q: How do I download all comments associated with *multiple* dockets?

Links to dockets used below:
* https://www.regulations.gov/docket/FDA-2009-N-0501
* https://www.regulations.gov/docket/EERE-2019-BT-STD-0036
* https://www.regulations.gov/docket/NHTSA-2019-0121

## Into a database only

Notice that the database is created automatically. If the database already existed, we would be appending to it, not replacing it

In [10]:
# you could add the csv_filename argument to export to both a database and a single CSV
my_dockets = ['FDA-2009-N-0501', 'EERE-2019-BT-STD-0036', 'NHTSA-2019-0121']

for docket_id in my_dockets:
    print(f"\n********************************\nSTARTING {docket_id}\n********************************")
    downloader.gather_comments_by_docket(docket_id, db_filename="my_database2.db")

print("\nDONE")


********************************
STARTING FDA-2009-N-0501
********************************
2021-05-27 12:06:01: Getting documents associated with docket FDA-2009-N-0501...

Found 9 documents...
2021-05-27 12:06:01: Inserting 9 records into database...
2021-05-27 12:06:01: Writing 9 records to document_headers_120601.csv...Done
2021-05-27 12:06:01: Removing any duplicates in the CSV...
2021-05-27 12:06:01: Done. Removed 0 duplicate rows from document_headers_120601.csv.
2021-05-27 12:06:01: Finished: approximately 9 documents collected

Done----------------

******************************
2021-05-27 12:06:01: Getting comments for document FDA-2009-N-0501-0011...

2021-05-27 12:06:01: Getting objectId for document FDA-2009-N-0501-0011...Got it (09000064845f4f11)
2021-05-27 12:06:02: Getting comment headers associated with document FDA-2009-N-0501-0011...

Found 0 comments...
2021-05-27 12:06:02: Finished: approximately 0 comments collected

Done getting all 0 comments for document FDA-2

## Look at the data

In [11]:
# look at the data

# See readme for database schema. The comment details (including its plain-text) are in comments_detail
import pandas as pd
import sqlite3

conn = sqlite3.connect("my_database2.db")
pd.read_sql_query("select docketId, count(*) as comments from comments_detail group by docketId", conn)

,docketId,comments
0,EERE-2019-BT-STD-0036,17
1,FDA-2009-N-0501,13
2,NHTSA-2019-0121,23


In [12]:
conn.close()

## Into one CSV per docket

If a path is specified and the directory does not exist, it will be created automatically.

In [13]:
# alternatively, you could specify the same csv_filename for all iterations and all the data will be appended to a single file
my_dockets = ['FDA-2009-N-0501', 'EERE-2019-BT-STD-0036', 'NHTSA-2019-0121']

for docket_id in my_dockets:
    print(f"\n********************************\nSTARTING {docket_id}\n********************************")
    downloader.gather_comments_by_docket(docket_id, csv_filename=f"mydockets/{docket_id}.csv")

print("\nDONE")


********************************
STARTING FDA-2009-N-0501
********************************
2021-05-27 12:11:53: Getting documents associated with docket FDA-2009-N-0501...

Found 9 documents...
2021-05-27 12:11:54: Writing 9 records to document_headers_121153.csv...Done
2021-05-27 12:11:54: Removing any duplicates in the CSV...
2021-05-27 12:11:54: Done. Removed 0 duplicate rows from document_headers_121153.csv.
2021-05-27 12:11:54: Finished: approximately 9 documents collected

Done----------------

******************************
2021-05-27 12:11:54: Getting comments for document FDA-2009-N-0501-0011...

2021-05-27 12:11:54: Getting objectId for document FDA-2009-N-0501-0011...Got it (09000064845f4f11)
2021-05-27 12:11:54: Getting comment headers associated with document FDA-2009-N-0501-0011...

Found 0 comments...
2021-05-27 12:11:54: Finished: approximately 0 comments collected

Done getting all 0 comments for document FDA-2009-N-0501-0011----------------

*************************

# Q: How do I download all comments associated with a document?

## Into both a database and a CSV

Same idea as above for dockets. Can remove one of the filename parameters to output to only a CSV or a database. If either file exists, the data are appended to the existing database/CSV.

In [15]:
downloader.gather_comments_by_document("FDA-2009-N-0501-0012", db_filename="mydb3.db", csv_filename="FDA_document.csv")

2021-05-27 12:18:19: Getting objectId for document FDA-2009-N-0501-0012...Got it (09000064847f0822)
2021-05-27 12:18:20: Getting comment headers associated with document FDA-2009-N-0501-0012...

Found 10 comments...
2021-05-27 12:18:21: Inserting 10 records into database...
2021-05-27 12:18:21: Writing 10 records to comment_headers_121820.csv...Done
2021-05-27 12:18:21: Removing any duplicates in the CSV...
2021-05-27 12:18:21: Done. Removed 0 duplicate rows from comment_headers_121820.csv.
2021-05-27 12:18:21: Finished: approximately 10 comments collected

Done getting comment IDs----------------

2021-05-27 12:18:21: Getting comments associated with document FDA-2009-N-0501-0012...

2021-05-27 12:18:21: Gathering details for 10 comments...
2021-05-27 12:18:23: Inserting 10 records into database...
2021-05-27 12:18:23: Writing 10 records to FDA_document.csv...Done
2021-05-27 12:18:23: Finished: 10 comments collected

Done getting all 10 comments for document FDA-2009-N-0501-0012------

# Q: How do I download all comments associated with *multiple* documents?

Links to documents used below:
* https://www.regulations.gov/document/FDA-2009-N-0501-0012
* https://www.regulations.gov/document/NHTSA-2019-0121-0001
* https://www.regulations.gov/document/EERE-2019-BT-STD-0036-0001

## Into both a database and one CSV per document

Same idea as above for dockets. You can remove one of the filename parameters to output to only a CSV or a database. In addition, you could write all data to a single CSV instead of one per document.

In [17]:
my_documents = ['FDA-2009-N-0501-0012', 'NHTSA-2019-0121-0001', 'EERE-2019-BT-STD-0036-0001']

for document_id in my_documents:
    print(f"\n********************************\nSTARTING {document_id}\n********************************")
    downloader.gather_comments_by_document(document_id, db_filename="mydb.db", csv_filename=f"mydocuments/{document_id}.csv")

print("\nDONE")


********************************
STARTING FDA-2009-N-0501-0012
********************************
2021-05-27 12:20:40: Getting objectId for document FDA-2009-N-0501-0012...Got it (09000064847f0822)
2021-05-27 12:20:40: Getting comment headers associated with document FDA-2009-N-0501-0012...

Found 10 comments...
2021-05-27 12:20:41: Inserting 10 records into database...
2021-05-27 12:20:41: Writing 10 records to comment_headers_122040.csv...Done
2021-05-27 12:20:41: Removing any duplicates in the CSV...
2021-05-27 12:20:41: Done. Removed 0 duplicate rows from comment_headers_122040.csv.
2021-05-27 12:20:41: Finished: approximately 10 comments collected

Done getting comment IDs----------------

2021-05-27 12:20:41: Getting comments associated with document FDA-2009-N-0501-0012...

2021-05-27 12:20:41: Gathering details for 10 comments...
2021-05-27 12:20:43: Inserting 10 records into database...
2021-05-27 12:20:43: Writing 10 records to mydocuments/FDA-2009-N-0501-0012.csv...Done
2021-

## Look at the data

In [20]:
# look at the data

# See readme for database schema. The comment details (including its plain-text) are in comments_detail
import pandas as pd
import sqlite3

conn = sqlite3.connect("mydb.db")
pd.read_sql_query("select commentOnDocumentId, count(*) as comments from comments_detail group by commentOnDocumentId", conn)

,commentOnDocumentId,comments
0,EERE-2019-BT-STD-0036-0001,17
1,FDA-2009-N-0501-0012,10
2,NHTSA-2019-0121-0001,23


In [21]:
conn.close()

# Q: How do I find out the number of [dockets/documents/comments] that would be returned by a query using some parameters?

Parameters need to be specified in accordance with the Regulations.gov API. See: https://open.gsa.gov/api/regulationsgov/#api-calls

Note about date formats: postedDate uses yyyy-mm-dd, while lastModifiedDate uses yyyy-mm-dd hh24:mi:ss (Eastern time zone)

In [25]:
# Get the number of comments associated with EPA dockets posted between 1/1/2021 and 1/7/2021
downloader.get_items_count("comments", params={'filter[postedDate][ge]': '2021-01-01',
                                               'filter[postedDate][le]': '2021-01-07',
                                               'filter[agencyId]': 'EPA'})

290

# Q: How do I download all [dockets/documents/comments] associated with some parameters?

## 1. Get the header information

In [26]:
params = {'filter[postedDate][ge]': '2021-01-01',
          'filter[postedDate][le]': '2021-01-07',
          'filter[agencyId]': 'EPA'}

downloader.gather_headers("comments", params, csv_filename="EPA_January_2021Week1_headers.csv")

Found 290 comments...

Enter outer loop (0 comments collected)...
    250 comments retrieved
    290 comments retrieved
2021-05-27 12:27:24: Writing 290 records to EPA_January_2021Week1.csv...Done
2021-05-27 12:27:24: Removing any duplicates in the CSV...
2021-05-27 12:27:24: Done. Removed 0 duplicate rows from EPA_January_2021Week1.csv.
2021-05-27 12:27:24: Finished: approximately 290 comments collected


## 2. Get the IDs associated with those headers

In [27]:
comment_ids = downloader.get_ids_from_csv("EPA_January_2021Week1_headers.csv", data_type="comments")
len(comment_ids)

290

## 3. Get the detail information

This is slower than the above because it gets each comment one at a time. You are more likely to hit your request limit (1000 requests/hr) in this step. 

Data is written out to a file every 500 rows by default, but this can be customized via the insert_every_n_rows parameter

In [28]:
downloader.gather_details("comments", comment_ids, csv_filename="EPA_January_2021Week1.csv")

2021-05-27 12:29:11: Gathering details for 290 comments...
(Requests left: 300)
(Requests left: 200)
2021-05-27 12:30:10: Writing 290 records to EPA_January_2021Week1.csv...Done
2021-05-27 12:30:10: Finished: 290 comments collected
